In [ ]:
import pandas as pd
import re

def testing_csv_processing(filepath):
    # Read the first two rows to create the header
    header_df = pd.read_csv(filepath, nrows=2, header=None)
    header_df.fillna('', inplace=True)
    header = header_df.apply(lambda x: ' '.join(map(str, x)).strip(), axis=0).tolist()

    # Ensure unique column names
    unique_header = []
    counts = {}
    for col in header:
        if col in counts:
            counts[col] += 1
            unique_header.append(f"{col}_{counts[col]}")
        else:
            counts[col] = 0
            unique_header.append(col)

    # Read the rest of the CSV data using the new header
    df = pd.read_csv(filepath, skiprows=2, header=None, names=unique_header)
    
    # Combine the first 5 columns to create unique column names, excluding 'nan' values
    df['combined'] = df[['Pod(L2)', 'Book(L3)', 'Underlier', 'Desk Strategy', 'Ticker']].astype(str).apply(lambda x: ' '.join(filter(lambda y: y != 'nan' and y != '', x)), axis=1)
    
    # Pivot the table to get the 'combined' columns with 'Delta' values
    pivot_df = df.pivot_table(index=None, columns='combined', values='Delta', aggfunc='first').reset_index(drop=True)
    
    # Fill down the values to handle hierarchical structure
    pivot_df.ffill(inplace=True)
    
    # Initialize a dictionary to store the final results
    result_dict = {}

    # List of traders to process
    traders = ['Beimnet', 'Bouchra', 'Eduardo', 'Felman']

    for trader in traders:
        # Columns to sum for BTC, ETH, and SOL (Core, Options, and RelativeValue)
        btc_core_col = f'{trader} Crypto BTC Core BTC'
        btc_options_cols = pivot_df.filter(regex=f'{trader} Crypto BTC Options $').columns
        btc_relative_col = f'{trader} Crypto BTC RelativeValue BTC'

        eth_core_col = f'{trader} Crypto ETH Core ETH'
        eth_options_cols = pivot_df.filter(regex=f'{trader} Crypto ETH Options $').columns
        eth_relative_col = f'{trader} Crypto ETH RelativeValue ETH'

        sol_core_col = f'{trader} Crypto SOL Core SOL'
        sol_options_cols = pivot_df.filter(regex=f'{trader} Crypto SOL Options $').columns
        sol_relative_col = f'{trader} Crypto SOL RelativeValue SOL'

        # Calculate the sum for BTC, ETH, and SOL deltas
        btc_core_sum = pivot_df[btc_core_col].sum() if btc_core_col in pivot_df else 0
        btc_options_sum = pivot_df[btc_options_cols[0]].sum() if len(btc_options_cols) > 0 else 0
        btc_relative_sum = pivot_df[btc_relative_col].sum() if btc_relative_col in pivot_df else 0
        btc_total_sum = btc_core_sum + btc_options_sum + btc_relative_sum

        eth_core_sum = pivot_df[eth_core_col].sum() if eth_core_col in pivot_df else 0
        eth_options_sum = pivot_df[eth_options_cols[0]].sum() if len(eth_options_cols) > 0 else 0
        eth_relative_sum = pivot_df[eth_relative_col].sum() if eth_relative_col in pivot_df else 0
        eth_total_sum = eth_core_sum + eth_options_sum + eth_relative_sum

        sol_core_sum = pivot_df[sol_core_col].sum() if sol_core_col in pivot_df else 0
        sol_options_sum = pivot_df[sol_options_cols[0]].sum() if len(sol_options_cols) > 0 else 0
        sol_relative_sum = pivot_df[sol_relative_col].sum() if sol_relative_col in pivot_df else 0
        sol_total_sum = sol_core_sum + sol_options_sum + sol_relative_sum

        # Initialize the Alts Delta bucket sum
        alts_core_sum = 0
        alts_options_sum = 0
        alts_relative_sum = 0

        # Iterate over all columns to find relevant Alts Delta columns
        for col in pivot_df.columns:
            if col.startswith(f'{trader} Crypto') and 'Delta' in col:
                parts = col.split()
                underlier = parts[-2]
                if 'Core' in col:
                    if underlier not in ['BTC', 'ETH', 'SOL']:
                        alts_core_sum += pivot_df[col].sum()
                elif 'Options' in col and len(parts) == 4:
                    if underlier not in ['BTC', 'ETH', 'SOL']:
                        alts_options_sum += pivot_df[col].sum()
                elif 'RelativeValue' in col:
                    if underlier not in ['BTC', 'ETH', 'SOL']:
                        alts_relative_sum += pivot_df[col].sum()

        alts_total_sum = alts_core_sum + alts_options_sum + alts_relative_sum

        # Calculate total delta for the trader
        trader_total_sum = btc_total_sum + eth_total_sum + sol_total_sum + alts_total_sum

        # Add the trader's deltas to the result dictionary
        result_dict[f'{trader} BTC Core Delta'] = btc_core_sum
        result_dict[f'{trader} BTC Options Delta'] = btc_options_sum
        result_dict[f'{trader} BTC RelativeValue Delta'] = btc_relative_sum
        result_dict[f'{trader} BTC Total Delta'] = btc_total_sum

        result_dict[f'{trader} ETH Core Delta'] = eth_core_sum
        result_dict[f'{trader} ETH Options Delta'] = eth_options_sum
        result_dict[f'{trader} ETH RelativeValue Delta'] = eth_relative_sum
        result_dict[f'{trader} ETH Total Delta'] = eth_total_sum

        result_dict[f'{trader} SOL Core Delta'] = sol_core_sum
        result_dict[f'{trader} SOL Options Delta'] = sol_options_sum
        result_dict[f'{trader} SOL RelativeValue Delta'] = sol_relative_sum
        result_dict[f'{trader} SOL Total Delta'] = sol_total_sum

        result_dict[f'{trader} Alts Core Delta'] = alts_core_sum
        result_dict[f'{trader} Alts Options Delta'] = alts_options_sum
        result_dict[f'{trader} Alts RelativeValue Delta'] = alts_relative_sum
        result_dict[f'{trader} Alts Total Delta'] = alts_total_sum

        result_dict[f'{trader} Total Delta'] = trader_total_sum

    # Process Novo separately
    novo_dict = {}

    # Columns to sum for BTC, ETH, and SOL (Core, ETF, Options)
    novo_btc_core_col = 'Novo Crypto BTC Core BTC'
    novo_btc_etf_cols = pivot_df.filter(regex='Novo Crypto BTC ETF $').columns
    novo_btc_options_cols = pivot_df.filter(regex='Novo Crypto BTC Options $').columns

    novo_eth_core_col = 'Novo Crypto ETH Core ETH'
    novo_eth_etf_cols = pivot_df.filter(regex='Novo Crypto ETH ETF $').columns
    novo_eth_options_cols = pivot_df.filter(regex='Novo Crypto ETH Options $').columns

    novo_sol_core_col = 'Novo Crypto SOL Core SOL'
    novo_sol_etf_cols = pivot_df.filter(regex='Novo Crypto SOL ETF $').columns
    novo_sol_ftx_col = 'Novo Crypto FTX_SOL SOL'
    novo_sol_options_cols = pivot_df.filter(regex='Novo Crypto SOL Options $').columns

    # Calculate the sum for BTC, ETH, and SOL deltas
    novo_btc_core_sum = pivot_df[novo_btc_core_col].sum() if novo_btc_core_col in pivot_df else 0
    novo_btc_etf_sum = pivot_df[novo_btc_etf_cols[0]].sum() if len(novo_btc_etf_cols) > 0 else 0
    novo_btc_options_sum = pivot_df[novo_btc_options_cols[0]].sum() if len(novo_btc_options_cols) > 0 else 0
    novo_btc_total_sum = novo_btc_core_sum + novo_btc_etf_sum + novo_btc_options_sum

    novo_eth_core_sum = pivot_df[novo_eth_core_col].sum() if novo_eth_core_col in pivot_df else 0
    novo_eth_etf_sum = pivot_df[novo_eth_etf_cols[0]].sum() if len(novo_eth_etf_cols) > 0 else 0
    novo_eth_options_sum = pivot_df[novo_eth_options_cols[0]].sum() if len(novo_eth_options_cols) > 0 else 0
    novo_eth_total_sum = novo_eth_core_sum + novo_eth_etf_sum + novo_eth_options_sum

    novo_sol_core_sum = pivot_df[novo_sol_core_col].sum() if novo_sol_core_col in pivot_df else 0
    novo_sol_etf_sum = pivot_df[novo_sol_etf_cols[0]].sum() if len(novo_sol_etf_cols) > 0 else 0
    novo_sol_ftx_sum = pivot_df[novo_sol_ftx_col].sum() if novo_sol_ftx_col in pivot_df else 0
    novo_sol_options_sum = pivot_df[novo_sol_options_cols[0]].sum() if len(novo_sol_options_cols) > 0 else 0
    novo_sol_total_sum = novo_sol_core_sum + novo_sol_etf_sum + novo_sol_ftx_sum + novo_sol_options_sum

    # Initialize the Alts Delta bucket sum for Novo
    novo_alts_sum = 0
    for col in pivot_df.columns:
        if col.startswith('Novo Crypto') and 'Delta' in col:
            parts = col.split()
            underlier = parts[-2]
            if 'Core' in col or 'Options' in col or 'RelativeValue' in col:
                if underlier not in ['BTC', 'ETH', 'SOL', 'BGCI', 'GDAM1']:
                    novo_alts_sum += pivot_df[col].sum()

    novo_total_sum = novo_btc_total_sum + novo_eth_total_sum + novo_sol_total_sum + novo_alts_sum

    # Add Novo's deltas to the result dictionary
    novo_dict['Novo BTC Core Delta'] = novo_btc_core_sum
    novo_dict['Novo BTC ETF Delta'] = novo_btc_etf_sum
    novo_dict['Novo BTC Options Delta'] = novo_btc_options_sum
    novo_dict['Novo BTC Total Delta'] = novo_btc_total_sum

    novo_dict['Novo ETH Core Delta'] = novo_eth_core_sum
    novo_dict['Novo ETH ETF Delta'] = novo_eth_etf_sum
    novo_dict['Novo ETH Options Delta'] = novo_eth_options_sum
    novo_dict['Novo ETH Total Delta'] = novo_eth_total_sum

    novo_dict['Novo SOL Core Delta'] = novo_sol_core_sum
    novo_dict['Novo SOL ETF Delta'] = novo_sol_etf_sum
    novo_dict['Novo SOL FTX Delta'] = novo_sol_ftx_sum
    novo_dict['Novo SOL Options Delta'] = novo_sol_options_sum
    novo_dict['Novo SOL Total Delta'] = novo_sol_total_sum

    novo_dict['Novo Alts Total Delta'] = novo_alts_sum
    novo_dict['Novo Total Delta'] = novo_total_sum

    # Sum all traders' deltas by asset grouping and total
    summary_dict = {key: 0 for key in result_dict.keys() if 'Total Delta' in key}
    summary_dict['GDLP ex-Novo Delta'] = 0
    summary_dict['GDLP ex-Novo ex-Alts Delta'] = 0
    for key, value in result_dict.items():
        if 'BTC Total Delta' in key:
            summary_dict['BTC Total Delta'] = summary_dict.get('BTC Total Delta', 0) + value
        if 'ETH Total Delta' in key:
            summary_dict['ETH Total Delta'] = summary_dict.get('ETH Total Delta', 0) + value
        if 'SOL Total Delta' in key:
            summary_dict['SOL Total Delta'] = summary_dict.get('SOL Total Delta', 0) + value
        if 'Alts Total Delta' in key:
            summary_dict['Alts Total Delta'] = summary_dict.get('Alts Total Delta', 0) + value
        if 'Total Delta' in key:
            summary_dict['GDLP ex-Novo Delta'] += value
            if 'Alts' not in key:
                summary_dict['GDLP ex-Novo ex-Alts Delta'] += value

    # Combine GDLP ex-Novo and Novo to get GDLP Delta
    gdlp_delta_dict = {
        'GDLP BTC Total Delta': summary_dict['BTC Total Delta'] + novo_dict.get('Novo BTC Total Delta', 0),
        'GDLP ETH Total Delta': summary_dict['ETH Total Delta'] + novo_dict.get('Novo ETH Total Delta', 0),
        'GDLP SOL Total Delta': summary_dict['SOL Total Delta'] + novo_dict.get('Novo SOL Total Delta', 0),
        'GDLP Alts Total Delta': summary_dict.get('Alts Total Delta', 0) + novo_dict.get('Novo Alts Total Delta', 0),
        'GDLP Total Delta': summary_dict['GDLP ex-Novo Delta'] + novo_dict.get('Novo Total Delta', 0)
    }

    # Combine all results into a single dictionary
    combined_dict = {**result_dict, **summary_dict, **novo_dict, **gdlp_delta_dict}

    # Create a single-row DataFrame from the combined dictionary
    final_df = pd.DataFrame(combined_dict, index=['Values'])

    print(final_df)

    return final_df

csv_filepath = '/mnt/data/Midday GDLP GRE Data 22072024.csv'

# Process the CSV file and print the results
testing_csv_processing(csv_filepath)


In [ ]:
import pandas as pd
from datetime import datetime
import re
import os

def process_single_csv(filepath):
    # Read the first two rows to create the header
    header_df = pd.read_csv(filepath, nrows=2, header=None)
    header_df.fillna('', inplace=True)
    header = header_df.apply(lambda x: ' '.join(map(str, x)).strip(), axis=0).tolist()
    
    # Ensure unique column names
    unique_header = []
    counts = {}
    for col in header:
        if col in counts:
            counts[col] += 1
            unique_header.append(f"{col}_{counts[col]}")
        else:
            counts[col] = 0
            unique_header.append(col)

    # Read the rest of the CSV data using the new header
    df = pd.read_csv(filepath, skiprows=2, header=None, names=unique_header)
    
    # Fill down values to handle hierarchical structure
    df.ffill(inplace=True)
    
    # Initialize an empty dictionary to store DataFrames for each group
    group_dfs = {}
    seen_underliers = {}
    total_sums = {}

    # Iterate through each row to construct columns dynamically
    for idx, row in df.iterrows():
        trader = row.iloc[0]
        if trader not in group_dfs:
            group_dfs[trader] = {}
            seen_underliers[trader] = set()
            total_sums[trader] = {"BTC": 0, "ETH": 0, "SOL": 0, "ALTS": 0, "TOTAL": 0}
        
        # Determine the column name based on the presence of underlier
        underlier = str(row.iloc[3])
        if pd.notna(row.iloc[3]):
            column_name = f"{trader} {underlier} Delta"
        else:
            keys = [str(row.iloc[0])]
            if pd.notna(row.iloc[1]):
                keys.append(str(row.iloc[1]))
            if pd.notna(row.iloc[2]):
                keys.append(str(row.iloc[2]))
            column_name = " ".join(keys) + " Delta"
        
        desk_strategy = " ".join(keys[:3])
        
        # Check if the underlier has been seen in this desk strategy
        if (desk_strategy, underlier) not in seen_underliers[trader]:
            seen_underliers[trader].add((desk_strategy, underlier))
            
            # Store the delta value in the corresponding column
            if column_name not in group_dfs[trader]:
                delta_value = row['Delta']
                group_dfs[trader][column_name] = delta_value
                
                # Update the total sums
                if underlier == "BTC":
                    total_sums[trader]["BTC"] += delta_value
                elif underlier == "ETH":
                    total_sums[trader]["ETH"] += delta_value
                elif underlier == "SOL":
                    total_sums[trader]["SOL"] += delta_value
                elif pd.notna(row.iloc[3]):
                    total_sums[trader]["ALTS"] += delta_value
                
                total_sums[trader]["TOTAL"] += delta_value

    # Add total columns to the DataFrame in the correct position
    for trader in group_dfs:
        # Convert the dictionary to a DataFrame
        temp_df = pd.DataFrame(group_dfs[trader], index=[0])
        
        # Create a new DataFrame with the total columns in the correct position
        total_df = pd.DataFrame({
            f"{trader} Total BTC Delta": [total_sums[trader]["BTC"]],
            f"{trader} Total ETH Delta": [total_sums[trader]["ETH"]],
            f"{trader} Total SOL Delta": [total_sums[trader]["SOL"]],
            f"{trader} Total ALTS Delta": [total_sums[trader]["ALTS"]],
            f"{trader} Total Delta": [total_sums[trader]["TOTAL"]]
        })
        
        # Find the position to insert the total columns
        core_delta_col = f"{trader} Crypto Core Delta"
        if core_delta_col in temp_df.columns:
            insert_pos = temp_df.columns.get_loc(core_delta_col) + 1
        else:
            insert_pos = len(temp_df.columns)
        
        # Insert the total columns into the correct position
        final_df = pd.concat([temp_df.iloc[:, :insert_pos], total_df, temp_df.iloc[:, insert_pos:]], axis=1)
        
        # Update the group_dfs with the new DataFrame
        group_dfs[trader] = final_df
            
        # Print all the individual DataFrames for each group
        for trader, df in group_dfs.items():
            print(f"Final DataFrame for {trader}:")
            print(df)
            print()

        return group_dfs

def testing_csv_processing(filepath):
    # Read the first two rows to create the header
    header_df = pd.read_csv(filepath, nrows=2, header=None)
    header_df.fillna('', inplace=True)
    header = header_df.apply(lambda x: ' '.join(map(str, x)).strip(), axis=0).tolist()

    # Ensure unique column names
    unique_header = []
    counts = {}
    for col in header:
        if col in counts:
            counts[col] += 1
            unique_header.append(f"{col}_{counts[col]}")
        else:
            counts[col] = 0
            unique_header.append(col)

    # Read the rest of the CSV data using the new header
    df = pd.read_csv(filepath, skiprows=2, header=None, names=unique_header)
    
    # Combine the first 5 columns to create unique column names, excluding 'nan' values
    df['combined'] = df[['Pod(L2)', 'Book(L3)', 'Underlier', 'Desk Strategy', 'Ticker']].astype(str).apply(lambda x: ' '.join(filter(lambda y: y != 'nan' and y != '', x)), axis=1)
    
    # Pivot the table to get the 'combined' columns with 'Delta' values
    pivot_df = df.pivot_table(index=None, columns='combined', values='Delta', aggfunc='first').reset_index(drop=True)
    
    # Fill down the values to handle hierarchical structure
    pivot_df.ffill(inplace=True)
    
    # Initialize a dictionary to store the final results
    result_dict = {}

    # List of traders to process
    traders = ['Beimnet', 'Bouchra', 'Eduardo', 'Felman']

    for trader in traders:
        # Columns to sum for BTC, ETH, and SOL (Core, Options, and RelativeValue)
        btc_core_col = f'{trader} Crypto BTC Core BTC'
        btc_options_cols = pivot_df.filter(regex=f'{trader} Crypto BTC Options $').columns
        btc_relative_col = f'{trader} Crypto BTC RelativeValue BTC'

        eth_core_col = f'{trader} Crypto ETH Core ETH'
        eth_options_cols = pivot_df.filter(regex=f'{trader} Crypto ETH Options $').columns
        eth_relative_col = f'{trader} Crypto ETH RelativeValue ETH'

        sol_core_col = f'{trader} Crypto SOL Core SOL'
        sol_options_cols = pivot_df.filter(regex=f'{trader} Crypto SOL Options $').columns
        sol_relative_col = f'{trader} Crypto SOL RelativeValue SOL'

        # Calculate the sum for BTC, ETH, and SOL deltas
        btc_core_sum = pivot_df[btc_core_col].sum() if btc_core_col in pivot_df else 0
        btc_options_sum = pivot_df[btc_options_cols].sum().sum() if btc_options_cols.shape[0] > 0 else 0
        btc_relative_sum = pivot_df[btc_relative_col].sum() if btc_relative_col in pivot_df else 0
        btc_total_sum = btc_core_sum + btc_options_sum + btc_relative_sum

        eth_core_sum = pivot_df[eth_core_col].sum() if eth_core_col in pivot_df else 0
        eth_options_sum = pivot_df[eth_options_cols].sum().sum() if eth_options_cols.shape[0] > 0 else 0
        eth_relative_sum = pivot_df[eth_relative_col].sum() if eth_relative_col in pivot_df else 0
        eth_total_sum = eth_core_sum + eth_options_sum + eth_relative_sum

        sol_core_sum = pivot_df[sol_core_col].sum() if sol_core_col in pivot_df else 0
        sol_options_sum = pivot_df[sol_options_cols].sum().sum() if sol_options_cols.shape[0] > 0 else 0
        sol_relative_sum = pivot_df[sol_relative_col].sum() if sol_relative_col in pivot_df else 0
        sol_total_sum = sol_core_sum + sol_options_sum + sol_relative_sum

        # Initialize the Alts Delta bucket sum
        alts_core_sum = 0
        alts_options_sum = 0
        alts_relative_sum = 0

        # Iterate over all columns to find relevant Alts Delta columns
        for col in pivot_df.columns:
            if col.startswith(f'{trader} Crypto') and 'Delta' in col:
                parts = col.split()
                underlier = parts[-2]
                if 'Core' in col:
                    if underlier not in ['BTC', 'ETH', 'SOL']:
                        alts_core_sum += pivot_df[col].sum()
                elif 'Options' in col and len(parts) == 4:
                    if underlier not in ['BTC', 'ETH', 'SOL']:
                        alts_options_sum += pivot_df[col].sum()
                elif 'RelativeValue' in col:
                    if underlier not in ['BTC', 'ETH', 'SOL']:
                        alts_relative_sum += pivot_df[col].sum()

        alts_total_sum = alts_core_sum + alts_options_sum + alts_relative_sum

        # Calculate total delta for the trader
        trader_total_sum = btc_total_sum + eth_total_sum + sol_total_sum + alts_total_sum

        # Add the trader's deltas to the result dictionary if they are non-zero
        if btc_core_sum != 0:
            result_dict[f'{trader} BTC Core Delta'] = btc_core_sum
        if btc_options_sum != 0:
            result_dict[f'{trader} BTC Options Delta'] = btc_options_sum
        if btc_relative_sum != 0:
            result_dict[f'{trader} BTC RelativeValue Delta'] = btc_relative_sum
        if btc_total_sum != 0:
            result_dict[f'{trader} BTC Total Delta'] = btc_total_sum

        if eth_core_sum != 0:
            result_dict[f'{trader} ETH Core Delta'] = eth_core_sum
        if eth_options_sum != 0:
            result_dict[f'{trader} ETH Options Delta'] = eth_options_sum
        if eth_relative_sum != 0:
            result_dict[f'{trader} ETH RelativeValue Delta'] = eth_relative_sum
        if eth_total_sum != 0:
            result_dict[f'{trader} ETH Total Delta'] = eth_total_sum

        if sol_core_sum != 0:
            result_dict[f'{trader} SOL Core Delta'] = sol_core_sum
        if sol_options_sum != 0:
            result_dict[f'{trader} SOL Options Delta'] = sol_options_sum
        if sol_relative_sum != 0:
            result_dict[f'{trader} SOL RelativeValue Delta'] = sol_relative_sum
        if sol_total_sum != 0:
            result_dict[f'{trader} SOL Total Delta'] = sol_total_sum

        if alts_core_sum != 0:
            result_dict[f'{trader} Alts Core Delta'] = alts_core_sum
        if alts_options_sum != 0:
            result_dict[f'{trader} Alts Options Delta'] = alts_options_sum
        if alts_relative_sum != 0:
            result_dict[f'{trader} Alts RelativeValue Delta'] = alts_relative_sum
        if alts_total_sum != 0:
            result_dict[f'{trader} Alts Total Delta'] = alts_total_sum

        if trader_total_sum != 0:
            result_dict[f'{trader} Total Delta'] = trader_total_sum

    # Sum all traders' deltas by asset grouping and total
    summary_dict = {key: 0 for key in result_dict.keys() if 'Total Delta' in key}
    summary_dict['GDLP ex-Novo Delta'] = 0
    summary_dict['GDLP ex-Novo ex-Alts Delta'] = 0
    for key, value in result_dict.items():
        if 'BTC Total Delta' in key:
            summary_dict['BTC Total Delta'] = summary_dict.get('BTC Total Delta', 0) + value
        elif 'ETH Total Delta' in key:
            summary_dict['ETH Total Delta'] = summary_dict.get('ETH Total Delta', 0) + value
        elif 'SOL Total Delta' in key:
            summary_dict['SOL Total Delta'] = summary_dict.get('SOL Total Delta', 0) + value
        elif 'Alts Total Delta' in key:
            summary_dict['Alts Total Delta'] = summary_dict.get('Alts Total Delta', 0) + value
        elif 'Total Delta' in key:
            summary_dict['GDLP ex-Novo Delta'] += value
            if 'Alts' not in key:
                summary_dict['GDLP ex-Novo ex-Alts Delta'] += value

    # Create DataFrames for the results and the summary
    result_df = pd.DataFrame(result_dict, index=['Deltas'])
    summary_df = pd.DataFrame(summary_dict, index=['Summary'])

    # Concatenate the results and summary DataFrames
    final_df = pd.concat([result_df, summary_df])

    print(final_df)

    final_df.to_csv('output/testing.csv')

    return final_df

csv_filepath = 'data/data.csv'

#process_single_csv(csv_filepath)
testing_csv_processing(csv_filepath)



#    result_df.to_csv('output/testing.csv')


In [ ]:
import pandas as pd
from datetime import datetime
import re
import os

def testing_csv_processing(filepath):
    # Read the first two rows to create the header
    header_df = pd.read_csv(filepath, nrows=2, header=None)
    header_df.fillna('', inplace=True)
    header = header_df.apply(lambda x: ' '.join(map(str, x)).strip(), axis=0).tolist()

    # Ensure unique column names
    unique_header = []
    counts = {}
    for col in header:
        if col in counts:
            counts[col] += 1
            unique_header.append(f"{col}_{counts[col]}")
        else:
            counts[col] = 0
            unique_header.append(col)

    # Read the rest of the CSV data using the new header
    df = pd.read_csv(filepath, skiprows=2, header=None, names=unique_header)
    
    # Combine the first 5 columns to create unique column names, excluding 'nan' values
    df['combined'] = df[['Pod(L2)', 'Book(L3)', 'Underlier', 'Desk Strategy', 'Ticker']].astype(str).apply(lambda x: ' '.join(filter(lambda y: y != 'nan' and y != '', x)), axis=1)
    
    # Pivot the table to get the 'combined' columns with 'Delta' values
    pivot_df = df.pivot_table(index=None, columns='combined', values='Delta', aggfunc='first').reset_index(drop=True)
    
    # Fill down the values to handle hierarchical structure
    pivot_df.ffill(inplace=True)
    
    # Initialize a dictionary to store the final results
    result_dict = {}

    # List of traders to process
    traders = ['Beimnet', 'Bouchra', 'Eduardo', 'Felman']

    for trader in traders:
        # Columns to sum for BTC, ETH, and SOL (Core, Options, and RelativeValue)
        btc_core_col = f'{trader} Crypto BTC Core BTC'
        btc_options_cols = pivot_df.filter(regex=f'{trader} Crypto BTC Options $').columns
        btc_relative_col = f'{trader} Crypto BTC RelativeValue BTC'

        eth_core_col = f'{trader} Crypto ETH Core ETH'
        eth_options_cols = pivot_df.filter(regex=f'{trader} Crypto ETH Options $').columns
        eth_relative_col = f'{trader} Crypto ETH RelativeValue ETH'

        sol_core_col = f'{trader} Crypto SOL Core SOL'
        sol_options_cols = pivot_df.filter(regex=f'{trader} Crypto SOL Options $').columns
        sol_relative_col = f'{trader} Crypto SOL RelativeValue SOL'

        # Calculate the sum for BTC, ETH, and SOL deltas
        btc_core_sum = pivot_df[btc_core_col].sum() if btc_core_col in pivot_df else 0
        btc_options_sum = pivot_df[btc_options_cols].sum().sum() if btc_options_cols.shape[0] > 0 else 0
        btc_relative_sum = pivot_df[btc_relative_col].sum() if btc_relative_col in pivot_df else 0
        btc_total_sum = btc_core_sum + btc_options_sum + btc_relative_sum

        eth_core_sum = pivot_df[eth_core_col].sum() if eth_core_col in pivot_df else 0
        eth_options_sum = pivot_df[eth_options_cols].sum().sum() if eth_options_cols.shape[0] > 0 else 0
        eth_relative_sum = pivot_df[eth_relative_col].sum() if eth_relative_col in pivot_df else 0
        eth_total_sum = eth_core_sum + eth_options_sum + eth_relative_sum

        sol_core_sum = pivot_df[sol_core_col].sum() if sol_core_col in pivot_df else 0
        sol_options_sum = pivot_df[sol_options_cols].sum().sum() if sol_options_cols.shape[0] > 0 else 0
        sol_relative_sum = pivot_df[sol_relative_col].sum() if sol_relative_col in pivot_df else 0
        sol_total_sum = sol_core_sum + sol_options_sum + sol_relative_sum

        # Initialize the Alts Delta bucket sum
        alts_core_sum = 0
        alts_options_sum = 0
        alts_relative_sum = 0

        # Iterate over all columns to find relevant Alts Delta columns
        for col in pivot_df.columns:
            if col.startswith(f'{trader} Crypto') and 'Delta' in col:
                parts = col.split()
                underlier = parts[-2]
                if 'Core' in col:
                    if underlier not in ['BTC', 'ETH', 'SOL']:
                        alts_core_sum += pivot_df[col].sum()
                elif 'Options' in col and len(parts) == 4:
                    if underlier not in ['BTC', 'ETH', 'SOL']:
                        alts_options_sum += pivot_df[col].sum()
                elif 'RelativeValue' in col:
                    if underlier not in ['BTC', 'ETH', 'SOL']:
                        alts_relative_sum += pivot_df[col].sum()

        alts_total_sum = alts_core_sum + alts_options_sum + alts_relative_sum

        # Calculate total delta for the trader
        trader_total_sum = btc_total_sum + eth_total_sum + sol_total_sum + alts_total_sum

        # Add the trader's deltas to the result dictionary if they are non-zero
        if btc_core_sum != 0:
            result_dict[f'{trader} BTC Core Delta'] = btc_core_sum
        if btc_options_sum != 0:
            result_dict[f'{trader} BTC Options Delta'] = btc_options_sum
        if btc_relative_sum != 0:
            result_dict[f'{trader} BTC RelativeValue Delta'] = btc_relative_sum
        if btc_total_sum != 0:
            result_dict[f'{trader} BTC Total Delta'] = btc_total_sum

        if eth_core_sum != 0:
            result_dict[f'{trader} ETH Core Delta'] = eth_core_sum
        if eth_options_sum != 0:
            result_dict[f'{trader} ETH Options Delta'] = eth_options_sum
        if eth_relative_sum != 0:
            result_dict[f'{trader} ETH RelativeValue Delta'] = eth_relative_sum
        if eth_total_sum != 0:
            result_dict[f'{trader} ETH Total Delta'] = eth_total_sum

        if sol_core_sum != 0:
            result_dict[f'{trader} SOL Core Delta'] = sol_core_sum
        if sol_options_sum != 0:
            result_dict[f'{trader} SOL Options Delta'] = sol_options_sum
        if sol_relative_sum != 0:
            result_dict[f'{trader} SOL RelativeValue Delta'] = sol_relative_sum
        if sol_total_sum != 0:
            result_dict[f'{trader} SOL Total Delta'] = sol_total_sum

        if alts_core_sum != 0:
            result_dict[f'{trader} Alts Core Delta'] = alts_core_sum
        if alts_options_sum != 0:
            result_dict[f'{trader} Alts Options Delta'] = alts_options_sum
        if alts_relative_sum != 0:
            result_dict[f'{trader} Alts RelativeValue Delta'] = alts_relative_sum
        if alts_total_sum != 0:
            result_dict[f'{trader} Alts Total Delta'] = alts_total_sum

        if trader_total_sum != 0:
            result_dict[f'{trader} Total Delta'] = trader_total_sum

    # Sum all traders' deltas by asset grouping and total
    summary_dict = {
        'BTC Total Delta': 0,
        'ETH Total Delta': 0,
        'SOL Total Delta': 0,
        'Alts Total Delta': 0,
        'GDLP ex-Novo Delta': 0,
        'GDLP ex-Novo ex-Alts Delta': 0
    }

    for key, value in result_dict.items():
        if 'BTC Total Delta' in key:
            summary_dict['BTC Total Delta'] += value
        elif 'ETH Total Delta' in key:
            summary_dict['ETH Total Delta'] += value
        elif 'SOL Total Delta' in key:
            summary_dict['SOL Total Delta'] += value
        elif 'Alts Total Delta' in key:
            summary_dict['Alts Total Delta'] += value
        elif 'Total Delta' in key:
            summary_dict['GDLP ex-Novo Delta'] += value
            if 'Alts' not in key:
                summary_dict['GDLP ex-Novo ex-Alts Delta'] += value

    # Add summary_dict values to result_dict
    result_dict.update(summary_dict)

    # Create a single-row DataFrame from the combined dictionary
    final_df = pd.DataFrame(result_dict, index=['Values'])

    print(final_df)

    final_df.to_csv('output/testing.csv', index=False)

    return final_df

csv_filepath = 'data/data.csv'

# Process the CSV file and print the results
testing_csv_processing(csv_filepath)
